# LAB 1: Ceaser Cipher


### **Objective**  
- To understand and implement the **Caesar Cipher**, one of the simplest and oldest encryption techniques.  
- To learn how to **encrypt** and **decrypt** text using a shift-based substitution cipher.  
- To demonstrate basic string manipulation and ASCII-based character shifting in Python.  

### **Input**  
- A plaintext message (e.g., `"HELLO WORLD"`)  
- A shift key (e.g., `3`)  

### **Output**  
- Encrypted ciphertext (e.g., `"KHOOR ZRUOG"`)  
- Decrypted plaintext (e.g., `"HELLO WORLD"`)  

In [1]:
def caesar_cipher(text, shift, encrypt=True):
    result = ""
    for char in text:
        if char.isalpha():  # Check if the character is a letter
            shift_amount = shift if encrypt else -shift
            ascii_offset = ord('A') if char.isupper() else ord('a')
            new_char = chr(((ord(char) - ascii_offset + shift_amount) % 26) + ascii_offset)
            result += new_char
        else:
            result += char  # Keep spaces and special characters unchanged
    return result

# Example usage
plaintext = "HELLO WORLD"
shift = 3

# Encrypting the text
ciphertext = caesar_cipher(plaintext, shift, encrypt=True)
print("Encrypted:", ciphertext)

# Decrypting the text
decrypted_text = caesar_cipher(ciphertext, shift, encrypt=False)
print("Decrypted:", decrypted_text)

Encrypted: KHOOR ZRUOG
Decrypted: HELLO WORLD



### **Inference**  
- The **Caesar Cipher** is a basic form of encryption where each letter in the plaintext is replaced by a letter a fixed number of positions down the alphabet.  
- It is **not secure** for modern cryptographic applications since it can be broken easily using brute-force attacks or frequency analysis.  
- It serves as a foundation for understanding more advanced encryption techniques.

# LAB 2: Playfair Cipher


### **Objective**  
- To implement the **Playfair Cipher**, a digraph substitution cipher.  
- To understand how **pairs of letters** are encrypted using a **5x5 matrix**.  
- To apply basic **string processing and matrix manipulation** in Python.  

### **Input**  
- A **plaintext message** (e.g., `"HELLO WORLD"`)  
- A **key** (e.g., `"SECRET"`)  

### **Output**  
- **Encrypted ciphertext** (e.g., `"GCNVQ YQTNF"`)  
- **Decrypted plaintext** (e.g., `"HELLO WORLD"`)  

### **Explanation**
1. **Matrix Generation**  
   - Removes duplicate letters in the **key**.  
   - Replaces **J with I** and fills in the **5×5 matrix**.  

2. **Text Preparation**  
   - Converts **plaintext** to **pairs of letters**.  
   - Adds **'X'** between duplicate letters and at the end if needed.  

3. **Encryption & Decryption Rules**  
   - **Same row** → Shift **right** (encrypt) / **left** (decrypt).  
   - **Same column** → Shift **down** (encrypt) / **up** (decrypt).  
   - **Rectangle rule** → Swap column positions.  

---

In [2]:
import itertools

def prepare_text(text):
    text = text.upper().replace("J", "I").replace(" ", "")  # Replace J with I
    prepared_text = ""
    i = 0
    while i < len(text):
        if i == len(text) - 1:  # If single letter remains, add 'X'
            prepared_text += text[i] + "X"
            break
        if text[i] == text[i + 1]:  # If duplicate letters, insert 'X'
            prepared_text += text[i] + "X"
            i += 1
        else:
            prepared_text += text[i] + text[i + 1]
            i += 2
    return prepared_text

def generate_playfair_matrix(key):
    key = "".join(dict.fromkeys(key.upper().replace("J", "I")))  # Remove duplicates
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
    matrix = list(key + "".join([ch for ch in alphabet if ch not in key]))
    matrix_2d = [matrix[i * 5:(i + 1) * 5] for i in range(5)]
    
    print("\nPlayfair Matrix:")
    for row in matrix_2d:
        print(" ".join(row))  # Print the matrix in a 5x5 format
    
    return matrix_2d

def find_position(matrix, letter):
    for row, col in itertools.product(range(5), repeat=2):
        if matrix[row][col] == letter:
            return row, col
    return None

def playfair_cipher(text, matrix, encrypt=True):
    processed_text = prepare_text(text)
    result = ""

    for i in range(0, len(processed_text), 2):
        a, b = processed_text[i], processed_text[i + 1]
        row_a, col_a = find_position(matrix, a)
        row_b, col_b = find_position(matrix, b)

        if row_a == row_b:  # Same row
            col_a = (col_a + 1) % 5 if encrypt else (col_a - 1) % 5
            col_b = (col_b + 1) % 5 if encrypt else (col_b - 1) % 5
        elif col_a == col_b:  # Same column
            row_a = (row_a + 1) % 5 if encrypt else (row_a - 1) % 5
            row_b = (row_b + 1) % 5 if encrypt else (row_b - 1) % 5
        else:  # Rectangle swap
            col_a, col_b = col_b, col_a

        result += matrix[row_a][col_a] + matrix[row_b][col_b]

    return result

# Example usage
key = "SECRET"
plaintext = "HELLO WORLD"

matrix = generate_playfair_matrix(key)

ciphertext = playfair_cipher(plaintext, matrix, encrypt=True)
print("\nEncrypted:", ciphertext)

decrypted_text = playfair_cipher(ciphertext, matrix, encrypt=False)
print("Decrypted:", decrypted_text)



Playfair Matrix:
S E C R T
A B D F G
H I K L M
N O P Q U
V W X Y Z

Encrypted: ISKYIQEWFQKC
Decrypted: HELXLOWORLDX


### **Inference**  
- The **Playfair Cipher** encrypts **pairs of letters** based on a 5×5 matrix generated from a keyword.  
- It replaces **'J' with 'I'** to fit into 25 letters.  
- It is more secure than the **Caesar Cipher** but still vulnerable to frequency analysis.  


# LAB 3: Hill Cipher


### **Objective**  
- To implement the **Hill Cipher**, a polygraphic substitution cipher.  
- To understand **matrix multiplication** in encryption and **modular inverse** in decryption.  
- To demonstrate the use of **linear algebra** for cryptographic purposes in Python.  

---

### **Input**  
- A **plaintext message** (e.g., `"HELP"`)  
- A **key matrix** (e.g., `[[6, 24], [1, 13]]`)  

### **Output**  
- **Encrypted ciphertext** (e.g., `"ZEBB"`)  
- **Decrypted plaintext** (e.g., `"HELP"`)  

---

### **Explanation**
1. **Convert text to numbers** (A = 0, B = 1, ..., Z = 25).  
2. **Encryption:**  
   - Arrange plaintext in **matrix form** and multiply with the **key matrix**.  
   - Take modulo **26** to stay within the alphabet.  
3. **Decryption:**  
   - Compute the **inverse of the key matrix mod 26**.  
   - Multiply with the ciphertext matrix and take modulo **26**.  

---

### **Example Output**
```
Key Matrix:
[6, 24]
[1, 13]

Encrypted: ZEBB
Decrypted: HELP
```

In [7]:
import numpy as np
import math

def text_to_numbers(text):
    return [ord(char) - ord('A') for char in text.upper()]

def numbers_to_text(numbers):
    return "".join(chr(num + ord('A')) for num in numbers)

def encrypt_hill(plaintext, key_matrix):
    n = len(key_matrix)
    plaintext = plaintext.upper().replace(" ", "")

    # Padding if needed
    while len(plaintext) % n != 0:
        plaintext += "X"

    plaintext_numbers = text_to_numbers(plaintext)
    plaintext_matrix = np.array(plaintext_numbers).reshape(-1, n).T

    key_matrix = np.array(key_matrix)

    # Encryption: C = (K * P) % 26
    ciphertext_matrix = (np.dot(key_matrix, plaintext_matrix) % 26).T
    ciphertext_numbers = ciphertext_matrix.flatten()
    
    return numbers_to_text(ciphertext_numbers)

def mod_inverse_matrix(matrix, mod):
    det = int(round(np.linalg.det(matrix)))  # Compute determinant
    det = det % mod  # Ensure within mod range

    # Check if determinant is invertible
    if math.gcd(det, mod) != 1:
        raise ValueError(f"Determinant {det} is not invertible mod {mod}. Choose a different key matrix.")

    det_inv = pow(det, -1, mod)  # Modular inverse of determinant

    adjugate = np.round(det * np.linalg.inv(matrix)).astype(int) % mod
    return (det_inv * adjugate) % mod

def decrypt_hill(ciphertext, key_matrix):
    key_matrix = np.array(key_matrix)
    inverse_key_matrix = mod_inverse_matrix(key_matrix, 26)

    n = len(key_matrix)
    ciphertext_numbers = text_to_numbers(ciphertext)
    ciphertext_matrix = np.array(ciphertext_numbers).reshape(-1, n).T

    # Decryption: P = (K_inv * C) % 26
    plaintext_matrix = (np.dot(inverse_key_matrix, ciphertext_matrix) % 26).T
    plaintext_numbers = plaintext_matrix.flatten()
    
    return numbers_to_text(plaintext_numbers)

# Valid Key Matrix
key_matrix = [[3, 3], [2, 5]]  # Valid because det = 9, gcd(9,26) = 1
plaintext = "HELP"

print("Key Matrix:")
for row in key_matrix:
    print(row)

ciphertext = encrypt_hill(plaintext, key_matrix)
print("\nEncrypted:", ciphertext)

decrypted_text = decrypt_hill(ciphertext, key_matrix)
print("Decrypted:", decrypted_text)


Key Matrix:
[3, 3]
[2, 5]

Encrypted: HIAT
Decrypted: HELP


### **Inference**  
- The **Hill Cipher** encrypts text using **matrix multiplication** over **mod 26**.  
- A valid key matrix must have an **inverse modulo 26** for decryption.  
- It provides stronger encryption than **Caesar** and **Playfair Ciphers**.  

---

# LAB 4: Vigenère Cipher 


### **Objective**  
- To implement the **Vigenère Cipher**, a polyalphabetic substitution cipher.  
- To understand how **key repetition** improves security over simple substitution ciphers like **Caesar Cipher**.  

---

### **Input**  
- **Plaintext**: `"HELLO"`  
- **Key**: `"KEY"`  

### **Output**  
- **Encrypted Ciphertext**: `"RIJVS"`  
- **Decrypted Plaintext**: `"HELLO"`  

---

### **Example Output**
```
Encrypted: RIJVS
Decrypted: HELLO
```

In [10]:
def vigenere_encrypt(plaintext, key):
    plaintext = plaintext.upper()
    key = key.upper()
    ciphertext = ""

    for i in range(len(plaintext)):
        shift = ord(key[i % len(key)]) - ord('A')
        new_char = chr((ord(plaintext[i]) - ord('A') + shift) % 26 + ord('A'))
        ciphertext += new_char

    return ciphertext

def vigenere_decrypt(ciphertext, key):
    ciphertext = ciphertext.upper()
    key = key.upper()
    plaintext = ""

    for i in range(len(ciphertext)):
        shift = ord(key[i % len(key)]) - ord('A')
        new_char = chr((ord(ciphertext[i]) - ord('A') - shift) % 26 + ord('A'))
        plaintext += new_char

    return plaintext

# Example Usage
plaintext = "THIS IS HELLO"
key = "KEY"

ciphertext = vigenere_encrypt(plaintext, key)
print("Encrypted:", ciphertext)

decrypted_text = vigenere_decrypt(ciphertext, key)
print("Decrypted:", decrypted_text)

Encrypted: DLGCXGCXFOPJY
Decrypted: THISTISTHELLO


### **Inference**  
- The Vigenère Cipher **shifts each letter** by an amount determined by a repeating **keyword**.  
- It is **stronger than the Caesar Cipher** because it uses multiple shift values instead of one.  
- **Breaking** it requires **frequency analysis** or **Kasiski examination**.  

---
---

# LAB 5: Vernam Cipher (One-Time Pad)


### **Objective**  
- To implement the **Vernam Cipher**, also known as the **One-Time Pad (OTP)**.  
- To demonstrate that when used with a truly random key, the Vernam Cipher is **unbreakable**.  

---

### **Input**  
- **Plaintext**: `"HELLO"`  
- **Key**: `"XMCKL"` (same length as plaintext)  

### **Output**  
- **Encrypted Ciphertext**: `"EQNVZ"`  
- **Decrypted Plaintext**: `"HELLO"`  


---

### **Explanation**
1. The key is **randomly generated** and must be **the same length as the plaintext**.
2. Each character is **XORed** with the corresponding character in the key.
3. The **same XOR operation** is used for **decryption** since:
   \[
   (P \oplus K) \oplus K = P
   \]

---


In [11]:
import random
import string

def generate_random_key(length):
    return ''.join(random.choice(string.ascii_uppercase) for _ in range(length))

def vernam_encrypt(plaintext, key):
    plaintext = plaintext.upper()
    key = key.upper()
    ciphertext = ""

    for i in range(len(plaintext)):
        encrypted_char = chr(((ord(plaintext[i]) - ord('A')) ^ (ord(key[i]) - ord('A'))) + ord('A'))
        ciphertext += encrypted_char

    return ciphertext

def vernam_decrypt(ciphertext, key):
    ciphertext = ciphertext.upper()
    key = key.upper()
    plaintext = ""

    for i in range(len(ciphertext)):
        decrypted_char = chr(((ord(ciphertext[i]) - ord('A')) ^ (ord(key[i]) - ord('A'))) + ord('A'))
        plaintext += decrypted_char

    return plaintext

# Example Usage
plaintext = "HELLO"
key = generate_random_key(len(plaintext))  # Generate a random key of same length

print("Generated Key:", key)

ciphertext = vernam_encrypt(plaintext, key)
print("Encrypted:", ciphertext)

decrypted_text = vernam_decrypt(ciphertext, key)
print("Decrypted:", decrypted_text)


Generated Key: QHLEK
Encrypted: XDAPE
Decrypted: HELLO


### **Inference** 
- The Vernam Cipher is a symmetric-key cipher where each character is XORed with a random key of the same length.  
- If the key is truly random, used only once, and kept secret, the cipher is theoretically unbreakable.  
- This is the only known encryption method with perfect secrecy, as proven by Claude Shannon.  


## LAB 6: Data Encryption Standard (DES)


### **Objective**  
- To implement **DES (Data Encryption Standard)** for encrypting and decrypting text using the **PyCryptodome** library.  
- To understand how block cipher encryption works using **Feistel network** principles.  

---

### **Input**  
- **Plaintext**: `"HELLOWOR"` (8 bytes required for DES)  
- **Key**: `"8CHARKEY"` (8-byte key required for DES)  

### **Output**  
- **Encrypted Ciphertext**: Hexadecimal representation of encrypted data  
- **Decrypted Plaintext**: `"HELLOWOR"`  

---

### **Explanation**
1. **Padding** ensures the plaintext is a multiple of **8 bytes**.
2. **ECB (Electronic Codebook) mode** is used for encryption.
3. The encrypted output is **converted to hexadecimal** for readability.
4. Decryption reverses the process, restoring the original plaintext.


In [18]:
from Crypto.Cipher import DES
import binascii

def pad(text):
    """Ensures text is 8 bytes long by padding with spaces."""
    while len(text) % 8 != 0:
        text += ' '
    return text

def des_encrypt(plaintext, key):
    cipher = DES.new(key.encode(), DES.MODE_ECB)  # Using Electronic Codebook (ECB) mode
    plaintext = pad(plaintext)  # Ensure 8-byte padding
    encrypted_text = cipher.encrypt(plaintext.encode())
    return binascii.hexlify(encrypted_text).decode()  # Convert to hexadecimal

def des_decrypt(ciphertext, key):
    cipher = DES.new(key.encode(), DES.MODE_ECB)
    decrypted_text = cipher.decrypt(binascii.unhexlify(ciphertext)).decode().rstrip()  # Remove extra spaces
    return decrypted_text

# Example Usage
plaintext = "HELLOWOR"  # Must be 8 bytes (or padded)
key = "8CHARKEY"  # DES requires an 8-byte key

ciphertext = des_encrypt(plaintext, key)
print("Encrypted:", ciphertext)

decrypted_text = des_decrypt(ciphertext, key)
print("Decrypted:", decrypted_text)


Encrypted: ec6d9932dbaa2942
Decrypted: HELLOWOR


### **Inference**  
- DES is a **symmetric-key block cipher** that operates on **64-bit blocks** with a **56-bit key** (8 bytes with parity).  
- It performs **16 rounds of Feistel transformations**, making it more secure than simple substitution ciphers.  
- DES is **now considered weak** due to **brute-force vulnerabilities**, but it laid the foundation for modern encryption standards like **AES**.  

---

## LAB 7: Advanced Encryption Standard (AES)



### **Objective**  
- To implement **AES (Advanced Encryption Standard)** for encrypting and decrypting text using the **PyCryptodome** library.  
- To understand how AES improves upon DES by using **128-bit block size**, **stronger key lengths (128, 192, 256 bits)**, and **substitution-permutation networks** instead of a Feistel structure.  

---

### **Input**  
- **Plaintext**: `"HELLOWORLD12345"` (AES works with 16-byte blocks)  
- **Key**: `"16BYTEKEY1234567"` (128-bit key)  

### **Output**  
- **Encrypted Ciphertext**: Hexadecimal representation of encrypted data  
- **Decrypted Plaintext**: `"HELLOWORLD12345"`  
---

### **Explanation**
1. **Padding** ensures that the plaintext is a multiple of **16 bytes**.
2. **ECB (Electronic Codebook) mode** is used for encryption (though **CBC mode** is preferred for security).
3. The encrypted output is **converted to hexadecimal** for readability.
4. Decryption reverses the encryption process, restoring the original plaintext.

---


In [19]:
from Crypto.Cipher import AES
import binascii

def pad(text):
    """Ensures text is a multiple of 16 bytes by adding padding."""
    while len(text) % 16 != 0:
        text += ' '
    return text

def aes_encrypt(plaintext, key):
    cipher = AES.new(key.encode(), AES.MODE_ECB)  # Using ECB mode (not recommended for real security)
    plaintext = pad(plaintext)  # Ensure 16-byte padding
    encrypted_text = cipher.encrypt(plaintext.encode())
    return binascii.hexlify(encrypted_text).decode()  # Convert to hexadecimal

def aes_decrypt(ciphertext, key):
    cipher = AES.new(key.encode(), AES.MODE_ECB)
    decrypted_text = cipher.decrypt(binascii.unhexlify(ciphertext)).decode().rstrip()  # Remove extra spaces
    return decrypted_text

# Example Usage
plaintext = "HELLOWORLD12345"  # 16 bytes
key = "16BYTEKEY1234567"  # 16-byte (128-bit) key

ciphertext = aes_encrypt(plaintext, key)
print("Encrypted:", ciphertext)

decrypted_text = aes_decrypt(ciphertext, key)
print("Decrypted:", decrypted_text)


Encrypted: fe787dfad0a74d37a6d579779d45480c
Decrypted: HELLOWORLD12345


### **Inference**  
- AES is a **symmetric block cipher** that works on **128-bit blocks** with **key sizes of 128, 192, or 256 bits**.  
- It is based on **Substitution-Permutation Networks (SPN)**, making it much **stronger and faster** than DES.  
- AES is widely used in **modern encryption standards**, including **SSL/TLS, disk encryption, and secure communications**.  

## LAB 8: RSA (Rivest-Shamir-Adleman) 

### **Objective**  
- To implement the **RSA algorithm** for encryption and decryption using the **PyCryptodome** library.  
- To understand how **asymmetric encryption** works using **public and private keys**.  

---

### **Input**  
- **Plaintext**: `"HELLO"`  
- **Key Size**: `1024-bit`  

### **Output**  
- **Public Key**: `(e, n)` (Used for encryption)  
- **Private Key**: `(d, n)` (Used for decryption)  
- **Encrypted Ciphertext**: Hexadecimal representation of the encrypted message  
- **Decrypted Plaintext**: `"HELLO"`  

---

### **Explanation**
1. **Key Generation**:  
   - A **1024-bit RSA key pair** is generated.  
   - The **public key** is used for encryption.  
   - The **private key** is used for decryption.  
   
2. **Encryption**:  
   - The plaintext is encrypted using the **public key**.  
   - The ciphertext is converted to **hexadecimal** for readability.  
   
3. **Decryption**:  
   - The ciphertext is decrypted using the **private key**, restoring the original message.  

---

In [20]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import binascii

# Generate RSA Key Pair
key = RSA.generate(1024)  # 1024-bit RSA key
public_key = key.publickey()  # Extract public key
private_key = key  # Private key

# Encrypt function
def rsa_encrypt(plaintext, public_key):
    cipher = PKCS1_OAEP.new(public_key)
    encrypted_text = cipher.encrypt(plaintext.encode())
    return binascii.hexlify(encrypted_text).decode()  # Convert to hexadecimal

# Decrypt function
def rsa_decrypt(ciphertext, private_key):
    cipher = PKCS1_OAEP.new(private_key)
    decrypted_text = cipher.decrypt(binascii.unhexlify(ciphertext)).decode()
    return decrypted_text

# Example Usage
plaintext = "HELLO"

# Encrypt with Public Key
ciphertext = rsa_encrypt(plaintext, public_key)
print("Encrypted:", ciphertext)

# Decrypt with Private Key
decrypted_text = rsa_decrypt(ciphertext, private_key)
print("Decrypted:", decrypted_text)

Encrypted: 387f273b1789a5ee6a02f7ca9765b9064a3d078bddb1d17a0f06599fd1bea3fcf3fb36e34fab15b9cc8a3db34c0137128a1858679c21d66bebad4d425ce28b4d0ecc09fd70df9be670075f85a5fc6f500e9ba87c047436ade2ecd939063ea2cba4097b1efc14de321f5999c0671309344ebf8e970e6104f97a2a51bc1b5ebd25
Decrypted: HELLO


### **Inference**  
- **RSA** is an **asymmetric encryption algorithm** that uses a **public key for encryption** and a **private key for decryption**.  
- It is based on the difficulty of **factoring large prime numbers**.  
- RSA is widely used in **secure communications (SSL/TLS), digital signatures, and cryptographic applications**.  

---

# LAB 9: Diffie-Hellman

## **LAB 9: Diffie-Hellman Key Exchange in Python**  

### **Objective**  
- To implement the **Diffie-Hellman Key Exchange Algorithm** in Python.  
- To understand how two parties can **securely generate a shared secret key** over an insecure channel without directly transmitting the key.  

---

### **Input**  
- **Public Prime (p)**: A large prime number.  
- **Primitive Root (g)**: A generator (primitive root modulo p).  
- **Private Keys (a, b)**: Secret numbers chosen by two users.  

### **Output**  
- **Public Keys (A, B)**: Computed using the formula:  
  \[
  A = g^a \mod p
  \]
  \[
  B = g^b \mod p
  \]
- **Shared Secret Key**:  
  \[
  S_A = B^a \mod p
  \]
  \[
  S_B = A^b \mod p
  \]
  Since \( S_A = S_B \), both parties will have the same shared secret.  

---

### **Explanation**
1. **Public Parameters (p, g)**:  
   - Chosen by both parties.  
   - **p** is a large prime number, and **g** is a primitive root modulo **p**.  

2. **Private Keys (a, b)**:  
   - Secret values chosen by Alice and Bob.  

3. **Public Keys (A, B)**:  
   - Computed using **modular exponentiation**:  
     \[
     A = g^a \mod p
     \]
     \[
     B = g^b \mod p
     \]

4. **Shared Secret Computation**:  
   - Both parties derive the same shared secret using their private keys:  
     \[
     S_A = B^a \mod p
     \]
     \[
     S_B = A^b \mod p
     \]
   - Since \( S_A = S_B \), they now have a common key.  

---

In [ ]:
import random

# Step 1: Public parameters (both parties agree on p and g)
p = 23  # Prime number
g = 5   # Primitive root mod p

# Step 2: Private keys (chosen secretly by Alice and Bob)
a = random.randint(1, p-1)  # Alice's private key
b = random.randint(1, p-1)  # Bob's private key

# Step 3: Compute public keys
A = pow(g, a, p)  # Alice computes A = g^a mod p
B = pow(g, b, p)  # Bob computes B = g^b mod p

# Step 4: Compute shared secret key
shared_secret_A = pow(B, a, p)  # Alice computes S = B^a mod p
shared_secret_B = pow(A, b, p)  # Bob computes S = A^b mod p

# Step 5: Verify both shared keys are equal
assert shared_secret_A == shared_secret_B

# Print values
print("Public Prime (p):", p)
print("Primitive Root (g):", g)
print("Alice's Private Key (a):", a)
print("Bob's Private Key (b):", b)
print("Alice's Public Key (A):", A)
print("Bob's Public Key (B):", B)
print("Shared Secret Key:", shared_secret_A)


Encrypted: 0c481b8386a5846824560058fb959e8843025dd133582999dd955ffc8835d804b9694ace7c2b3de6540666f2fa82af523acd8e80a3e689e7dac15925c52d5d156b58cd774c84ff8d37646be786df40985c3bac828913ff847f75e67bf2b98de3940b6cb5dc9f7151fe4bf978f127ceec14a38d27566fc1f13c0775ec4e95e947
Decrypted: HELLO


### **Inference**  
- The **Diffie-Hellman Key Exchange** allows two parties to securely establish a shared secret key without directly sharing it.  
- The security of Diffie-Hellman is based on the **difficulty of computing discrete logarithms**.  
- Once the shared secret is generated, it can be used for **symmetric encryption (e.g., AES)**.  
